In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from mpl_axes_aligner import align
import numpy as np
import pandas as pd
import h5py
import math
import torch
from torchviz import make_dot
from JPwaptool import JPwaptool
import wf_func as wff
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.size'] = 12
plt.rcParams['lines.markersize'] = 4.0
plt.rcParams['lines.linewidth'] = 1.0
plt.rcParams['mathtext.fontset'] = 'cm'

In [2]:
t = np.array([0, 1, 2, 3])
a = np.array([0.5, 0.5, 0.5, 1])
b1 = np.array([0.5, 0.5, 1, 0.5])
b2 = np.array([1, 0.5, 0.5, 0.5])
fig = plt.figure()
gs = gridspec.GridSpec(2, 2, figure=fig, left=0.12, right=0.95, top=0.99, bottom=0.15, wspace=0.4, hspace=0.5)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax0.bar(t, a, color='r')
ax0.set_xlabel('$a$', fontsize=20)
ax0.set_ylabel('$weight\_a$', fontsize=20)
ax0.set_ylim(-0.1, 1.1)
ax0.set_xticks(t)
ax1.bar(t, b1, color='b')
ax1.set_xlabel('$b1$', fontsize=20)
ax1.set_ylabel('$weight\_b1$', fontsize=20)
ax1.set_xticks(t)
ax2.bar(t, b2, color='b')
ax2.set_xlabel('$b2$', fontsize=20)
ax2.set_ylabel('$weight\_b2$', fontsize=20)
ax2.set_xticks(t)
ax3.text(0, 0, '$L_{2}\ =\ 0.25$', fontsize=25)
ax3.set_xlim(0, 5)
ax3.set_ylim(-1, 1)
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.spines['bottom'].set_visible(False)
ax3.spines['left'].set_visible(False)
ax3.get_xaxis().set_visible(False)
ax3.get_yaxis().set_visible(False)
fig.savefig('img/tab.png')
fig.clf()
plt.close(fig)

In [3]:
t = np.array([0, 1, 2, 3])
a = np.array([3, 7, 0, 0])
b = np.array([0, 0, 5, 5])
fig = plt.figure()
ax = fig.add_subplot(111)
ax.vlines(t, 0, a, label='$weight\_a$', color = 'b', linewidth = 6)
ax.set_xlim(-0.5, 3.5)
ax.set_ylim(0, 10)
ax.tick_params(labelsize=20)
ax.set_xlabel('$a$', fontsize=30)
ax.set_ylabel('$weight$', fontsize=30)
ax.set_xticks(t)
ax.legend(prop={'size': 20})
fig.savefig('img/a.png', bbox_inches='tight')
fig.clf()
plt.close(fig)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.vlines(t, 0, b, label='$weight\_b$', color = 'r', linewidth = 6)
ax.set_xlim(-0.5, 3.5)
ax.set_ylim(0, 10)
ax.tick_params(labelsize=20)
ax.set_xlabel('$b$', fontsize=30)
ax.set_ylabel('$weight$', fontsize=30)
ax.set_xticks(t)
ax.legend(prop={'size': 20})
fig.savefig('img/b.png', bbox_inches='tight')
fig.clf()
plt.close(fig)
t = np.append(np.array([0]), t)
a = np.append(np.array([0]), np.cumsum(a / a.sum()))
b = np.append(np.array([0]), np.cumsum(b / b.sum()))
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hlines(0, -1, 4, color='k')
al = ax.step(t, a, label='$cdf\ weight\_a$', color = 'b', where='post', linewidth = 3)
bl = ax.step(t, b, label='$cdf\ weight\_b$', color = 'r', where='post', linewidth = 3)
ax.fill_between(np.repeat(t, 2)[2:], np.zeros(len(np.repeat(t, 2)[2:])), np.repeat(b, 2)[1:-1], facecolor='C1', alpha=0.1, color=None)
ax.fill_between(np.repeat(t, 2)[2:], np.repeat(a, 2)[1:-1], np.repeat(b, 2)[1:-1], facecolor='blue', alpha=0.5, color=None, label='$W-dist Area$')
ax.set_xlim(-0.5, 3.5)
ax.set_ylim(-0.1, 1.1)
ax.tick_params(labelsize=20)
ax.grid()
ax.text(1.1, 0.7, '$W-dist\ Area$', fontsize=25)
ax.set_xlabel('$a\ or\ b$', fontsize=30)
ax.set_ylabel('$normalized\ weight$', fontsize=30)
ax.legend()
fig.savefig('img/ab.png', bbox_inches='tight')
fig.clf()
plt.close(fig)

In [4]:
stream = JPwaptool(1029, 150, 600, 7, 15)
spe_pre = wff.read_model('spe-jinp.h5')
with h5py.File('/srv/waveform-analysis/dataset/jinp/ztraining-0.h5', 'r', libver='latest', swmr=True) as ipt:
    ent = ipt['Waveform'][:]
    tru = ipt['GroundTruth'][:]

In [7]:
i = 1
stream.Calculate(ent[i]['Waveform'])
cid = ent[i]['ChannelID']
eid = ent[i]['EventID']
truth = np.sort(tru[(tru['EventID'] == eid) & (tru['ChannelID'] == cid)], kind='stable', order=['EventID', 'ChannelID', 'RiseTime'])
wave = (ent[i]['Waveform'] - stream.ChannelInfo.Pedestal) * spe_pre[ent[i]['ChannelID']]['epulse']

In [8]:
df = pd.DataFrame(truth)
df = df.rename(columns={'RiseTime':'HitTime'})
df

,EventID,ChannelID,HitTime,Charge
0,1,1,294,142.897266
1,1,1,294,185.622822
2,1,1,300,184.157551
3,1,1,310,190.715757
4,1,1,313,151.900865
5,1,1,325,191.807541
6,1,1,351,154.046201
7,1,1,369,124.298918


In [11]:
truth['RiseTime'].dtype

dtype('int16')

In [20]:
fig = plt.figure()
fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, 1029, color='g', label='Threshold')
ax2.vlines(truth['RiseTime'].min(), 0, wave.sum(), color='r', label='Record')
print(truth['RiseTime'].min(), wave.sum())
ax.set_xlim(100, 500)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel('$t/\mathrm{ns}$')
ax.set_ylabel('$Voltage/\mathrm{mV}$')
ax2.set_ylabel('$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
ax2.annotate('$(271\mathrm{ns}, 2488.0\mathrm{mV}\cdot\mathrm{ns})$', xy=(truth['RiseTime'].min(), wave.sum()/2), xytext=(truth['RiseTime'].min()-150, wave.sum()*2/3), arrowprops=dict(facecolor='k', shrink=0.1, width=0.1, headwidth=2))
align.yaxes(ax, 0, ax2, 0)
fig.savefig('img/previous.png', bbox_inches='tight')
fig.clf()
plt.close(fig)

271 2488.0000000000264


In [21]:
fig = plt.figure()
fig.tight_layout()
ax = fig.add_subplot(111)
ax.plot(wave, label='Waveform')
ax.set_xlim(100, 500)
ax.set_xlabel('$t/\mathrm{ns}$')
ax.set_ylabel('$Voltage/\mathrm{mV}$')
ax.legend()
fig.savefig('img/wave.png', bbox_inches='tight')
fig.clf()
plt.close(fig)

In [22]:
fig = plt.figure()
fig.tight_layout()
ax = fig.add_subplot(111)
ax.vlines(truth['RiseTime'], 0, truth['Charge'], color='r', label='Charge')
ax.set_xlabel('$t/\mathrm{ns}$')
ax.set_ylabel('$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
ax.set_xlim(0, len(wave))
ax.legend()
fig.savefig('img/charge.png', bbox_inches='tight')
fig.clf()
plt.close(fig)

In [23]:
fig = plt.figure()
fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, 1029, color='g', label='Threshold')
ax2.vlines(truth['RiseTime'], 0, truth['Charge'], color='r', label='Charge')
ax.set_xlim(100, 500)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel('$t/\mathrm{ns}$')
ax.set_ylabel('$Voltage/\mathrm{mV}$')
ax2.set_ylabel('$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('img/goal.png', bbox_inches='tight')
fig.clf()
plt.close(fig)

In [24]:
print(wave.sum())
print(truth['Charge'][truth['Charge'] > 0].sum())

2488.0000000000264
2388.0511151143314


In [25]:
t = np.load('/srv/waveform-analysis/dataset/jinp/Charge/ztraining-Channel05/testing_record_2020-08-02_08:22:40.npz')['arr_0']

In [26]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(np.arange(1, len(t)+1), t, label='W-distance', color='C1')
ax.set_xlabel('$epoch$')
ax.set_ylabel('$Wasserstein\ Distance/\mathrm{ns}$')
ax.legend()
ax.grid()
fig.savefig('img/epoch.png')
fig.clf()
plt.close(fig)

In [27]:
pet, pwe = wff.threshold(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(wave, label='Waveform')
ax.hlines(spe_pre[cid]['thres'], 0, 1029, color='g', label='Threshold')
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.set_xlim(100, 500)
ax2.annotate('', xy=(pet.mean(), pwe.max()*0.7), xytext=(pet.mean()+pet.ptp()/2, pwe.max()*0.7), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel('$t/\mathrm{ns}$')
ax.set_ylabel('$Voltage/\mathrm{mV}$')
ax2.set_ylabel('$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('img/threshold.png', bbox_inches='tight')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid, mode='Charge')

PEnum is 17
truth RiseTime = [271 272 276 278 283 284 285 287 290 293 301 306 316 331 373 489], Weight = [256.22730803  70.78286561 150.13576328 148.26017169 262.0190966
  90.73764185 166.07706861  90.57009829 154.97070323 224.36935435
  78.89552354 138.39822323 108.31198849 112.6873069  208.81511771
 126.79288372]
truth Resi-norm = 34.8072334795751
RiseTime = [269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286
 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304
 305 306 307 308 309 315 316 317 318 331 332 372 373 374 375 376 489 490], Weight = [ 20.03634467  39.59860398  53.04765725  56.71558087  51.82501605
  49.37973363  50.60237484  57.93822208  61.6061457   60.38350449
  55.49293967  50.60237484  54.27029846  70.16463415  89.72689345
 101.95330552 101.95330552  95.84009949  82.39104621  73.83255777
  72.60991656  71.38727535  72.60991656  76.27784018  76.27784018
  65.27406932  51.82501605  44.48916881  32.26275674  28.59483312
  26.149550

In [15]:
pet = np.array(stream.ChannelInfo.PeakLoc) - spe_pre[cid]['peak_c']
pwe = np.array(stream.ChannelInfo.PeakAmp) / spe_pre[cid]['spe'].max()
pwe = pwe[pet >= 0]; pet = pet[pet >= 0]
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(wave, label='Waveform')
ax.hlines(spe_pre[cid]['thres'], 0, 1029, color='g', label='Threshold')
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=1.5)
ax.set_xlim(100, 500)
loc = np.array(stream.ChannelInfo.PeakLoc)
amp = np.array(stream.ChannelInfo.PeakAmp)
for i in range(len(np.array(stream.ChannelInfo.PeakLoc))):
    ax.annotate('', xy=(loc[i], amp[i]+5), xytext=(loc[i], amp[i]+15), arrowprops=dict(facecolor='k', shrink=0.01, width=0.5, headwidth=1))
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel('$t/\mathrm{ns}$')
ax.set_ylabel('$Voltage/\mathrm{mV}$')
ax2.set_ylabel('$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('img/findpeak.png', bbox_inches='tight')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid, mode='Charge')

PEnum is 17
truth RiseTime = [271 272 276 278 283 284 285 287 290 293 301 306 316 331 373 489], Weight = [256.22730803  70.78286561 150.13576328 148.26017169 262.0190966
  90.73764185 166.07706861  90.57009829 154.97070323 224.36935435
  78.89552354 138.39822323 108.31198849 112.6873069  208.81511771
 126.79288372]
truth Resi-norm = 34.8072334795751
RiseTime = [111 272 277 284 292 298 306 315 331 341 374 388 489], Weight = [ 21.60987861 295.89912097 321.41439933 531.91544579 397.96023441
 149.18627041 181.08036835 117.29217246 104.53453328  34.36751779
 193.83800753  27.9886982  110.91335287]
wdist = 4.017335197822407,cdiff = 99.9488848856954
Resi-norm = 70.55598542154097


In [16]:
pet, pwe = wff.waveformfft(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(wave, label='Waveform')
ax.hlines(spe_pre[cid]['thres'], 0, 1029, color='g', label='Threshold')
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.set_xlim(100, 500)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel('$t/\mathrm{ns}$')
ax.set_ylabel('$Voltage/\mathrm{mV}$')
ax2.set_ylabel('$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('img/fftrans.png', bbox_inches='tight')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid, mode='Charge')

PEnum is 17
truth RiseTime = [271 272 276 278 283 284 285 287 290 293 301 306 316 331 373 489], Weight = [256.22730803  70.78286561 150.13576328 148.26017169 262.0190966
  90.73764185 166.07706861  90.57009829 154.97070323 224.36935435
  78.89552354 138.39822323 108.31198849 112.6873069  208.81511771
 126.79288372]
truth Resi-norm = 34.8072334795751
RiseTime = [263 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285
 286 287 290 291 292 293 294 299 300 301 304 305 306 307 308 314 315 316
 317 318 330 331 332 333 371 372 373 374 375 487 488 489 490 491], Weight = [ 18.89170056  51.95807347  81.67357861  86.92015698  69.85237306
  46.46479476  34.75758364  41.62060544  58.21722722  67.50044279
  58.38437745  35.80445106  18.57628598  25.89502331  62.0227676
 110.6537148  143.91406482 140.47282059 100.37446995  46.05134565
  38.37222329  78.49955417  99.62519211  87.69363104  50.09125426
  20.84531052  24.71211089  16.66133588  19.56948302  36.31675993
  45.64240548  39.97

In [17]:
pet, pwe = wff.lucyddm(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(wave, label='Waveform')
ax.hlines(spe_pre[cid]['thres'], 0, 1029, color='g', label='Threshold')
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.set_xlim(100, 500)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel('$t/\mathrm{ns}$')
ax.set_ylabel('$Voltage/\mathrm{mV}$')
ax2.set_ylabel('$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
fig.savefig('img/lucyddm.png', bbox_inches='tight')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid, mode='Charge')

PEnum is 17
truth RiseTime = [271 272 276 278 283 284 285 287 290 293 301 306 316 331 373 489], Weight = [256.22730803  70.78286561 150.13576328 148.26017169 262.0190966
  90.73764185 166.07706861  90.57009829 154.97070323 224.36935435
  78.89552354 138.39822323 108.31198849 112.6873069  208.81511771
 126.79288372]
truth Resi-norm = 34.8072334795751
RiseTime = [270 271 272 273 274 275 276 277 278 279 281 282 283 284 285 286 287 288
 289 290 291 292 293 294 300 301 305 306 307 315 316 317 330 331 332 372
 373 374 488 489 490], Weight = [ 42.11502401 185.47167698  95.40050534  29.23257622  19.14818272
  30.00748753  69.95923194 100.05923537  68.53987916  28.40848996
  19.80738476  50.90970357 136.59895746 200.84013603 141.65479721
  72.48230455  38.25103667  28.41809479  30.29773312  47.35054324
  74.60066642 103.94031376  87.26098822  44.23204211  18.66420912
  23.6744544   37.83840575  65.13310344  27.56286519  28.12672881
  61.43394255  25.10506744  22.88159826  51.85412511  25.124424

In [18]:
pet, pwe0 = wff.xiaopeip(wave, spe_pre[cid])
pet, pwe0 = wff.clip(pet, pwe0, 0.1)

In [19]:
pwe = pwe0 * spe_pre[cid]['spe'].sum()
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid, mode='Charge', full=True)

PEnum is 17
truth RiseTime = [271 272 276 278 283 284 285 287 290 293 301 306 316 331 373 489], Weight = [256.22730803  70.78286561 150.13576328 148.26017169 262.0190966
  90.73764185 166.07706861  90.57009829 154.97070323 224.36935435
  78.89552354 138.39822323 108.31198849 112.6873069  208.81511771
 126.79288372]
truth Resi-norm = 34.8072334795751
RiseTime = [271 272 276 277 278 280 283 284 286 289 290 293 301 306 316 331 373 489], Weight = [260.87802881  65.26760515 133.93157741  54.67775208  96.8138964
  29.17937194 200.95889878 230.16553555 159.17313741  21.94858217
 142.94558632 226.90802868  67.01221323 135.86870282 105.18002571
  98.61035692 210.40607498 110.93348178]
wdist = 1.5062452516856446,cdiff = -37.1922589851265
Resi-norm = 35.02818278545695


In [12]:
s = ['$\mathrm{FindPeak}$', '$\mathrm{Threshold}$', '$\mathrm{FFT}$', '$\mathrm{LucyDDM}$', '$\mathrm{CNN}$', '$\mathrm{Fitting}$']
x = np.arange(0, 6)
y = np.array([10.71, 3.67, 1.45, 0.93, 0.26, 0.504])
dy = np.array([[1.32, 1.80, 0.86, 0.49, 0.065, 0.16], [24.06, 6.11, 2.12, 1.41, 0.425, 0.87]])
dy[0, :] = y - dy[0, :]; dy[1, :] = dy[1, :] - y
fig = plt.figure()
ax = fig.add_subplot(111)
ax.bar(x, y, color='b')
ax.set_ylim(0, 26)
ax.set_ylabel('$\mathrm{Wasserstein}\ \mathrm{Distance}/\mathrm{ns}$')
ax.set_xticks(x)
ax.set_xticklabels(s)
ax.errorbar(x, y, yerr=dy, fmt='o', ecolor='r', color='b', elinewidth=2, capsize=4)
fig.savefig('img/summarycharge.png', bbox_inches='tight')
fig.clf()
plt.close(fig)

In [22]:
s = ['$\mathrm{FindPeak}$', '$\mathrm{Threshold}$', '$\mathrm{FFT}$', '$\mathrm{LucyDDM}$', '$\mathrm{CNN}$', '$\mathrm{Fitting}$']
x = np.arange(0, 6)
y = np.array([11.62, 4.69, 3.08, 2.74, 0.76, 2.46])
dy = np.array([[2.13, 1.78, 1.04, 0.77, 0.014, 0.57], [25.16, 8.51, 5.65, 5.24, 1.78, 4.83]])
dy[0, :] = y - dy[0, :]; dy[1, :] = dy[1, :] - y
fig = plt.figure()
ax = fig.add_subplot(111)
ax.bar(x, y, color='b')
ax.set_ylim(0, 26)
ax.set_ylabel('$\mathrm{Wasserstein}\ \mathrm{Distance}/\mathrm{ns}$')
ax.set_xticks(x)
ax.set_xticklabels(s)
ax.errorbar(x, y, yerr=dy, fmt='o', ecolor='r', color='b', elinewidth=2, capsize=4)
fig.savefig('img/summarypenum.png', bbox_inches='tight')
fig.clf()
plt.close(fig)

In [36]:
with h5py.File('spe-jinp.h5', 'r', libver='latest', swmr=True) as speFile:
    s = speFile['SinglePE'][:]
    a = s[s['ChannelID'] == 0]['speWf']
    a = a[:min(10, len(a))]

In [41]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(a.T*-1)
ax.set_xlabel('$t/\mathrm{ns}$')
ax.set_ylabel('$Voltage/\mathrm{mV}$')
fig.savefig('img/spewaves.png', bbox_inches='tight')
fig.clf()
plt.close(fig)